[Chapter 4] Setting a Strong Baseline Forecast

3. assessing the forecastability of a ts

In [2]:
import os
from pathlib import Path
from IPython.display import display, HTML
from tqdm.autonotebook import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd
from src.imputation.interpolation import SeasonalInterpolation

from statsforecast.core import StatsForecast
from utilsforecast.evaluation import evaluate
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    SimpleExponentialSmoothing,
    AutoETS,
    AutoTheta,
    Theta,
    OptimizedTheta,

)
from datasetsforecast.losses import *

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

C:\Users\user\AppData\Local\Temp\ipykernel_25440\2049660368.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [10]:
# read data

train_df = pd.read_parquet("./data/london_smart_meters/preprocessed/selected_blocks_train_missing_imputed.parquet")
train_df = train_df[['LCLid',"timestamp","energy_consumption","frequency"]]

In [15]:
# read and select data (in compact form)

# read raw data
lclid_acorn_map = pd.read_pickle("./data/london_smart_meters/preprocessed/london_smart_meters_lclid_acorn_map.pkl")

# stratification based on acorn calssification
affluent_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Affluent", ["LCLid",'file']]
adversity_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Adversity", ["LCLid",'file']]
comfortable_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Comfortable", ["LCLid",'file']]

# sampling: randomly select 10 households for each calssification (30 in total) for easier illustration
selected_households = pd.concat(
    [
        affluent_households.sample(10, random_state=76),
        comfortable_households.sample(10, random_state=76),
        adversity_households.sample(10, random_state=76),
    ]
)
selected_households['block']=selected_households.file.str.split("_", expand=True).iloc[:,1].astype(int)

# extracting the paths to the different blocks and extracting the starting and ending blocks
path_blocks = [
    (p, *list(map(int, p.name.split("_")[5].split(".")[0].split("-"))))
    for p in Path("data/london_smart_meters/preprocessed").glob(
        "london_smart_meters_merged_block*"
    )
]

household_df_l = []
for path, start_b, end_b in tqdm(path_blocks):
    block_df = pd.read_parquet(path, engine='fastparquet')
    selected_households['block'].between
    mask = selected_households['block'].between(start_b, end_b)
    lclids = selected_households.loc[mask, "LCLid"]
    household_df_l.append(block_df.loc[block_df.LCLid.isin(lclids)])

block_df = pd.concat(household_df_l)
del household_df_l
block_df.head(2)

  0%|          | 0/14 [00:00<?, ?it/s]

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
3916,MAC000193,2012-01-01,30min,"[0.368, 0.386, 0.17, 0.021, 0.038, 0.038, 0.02...",37872,ToU,ACORN-D,Affluent,block_7,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[229, 229, 238, 238, 229, 229, 231, 231, 227, ...","[12.12, 12.12, 12.59, 12.59, 12.45, 12.45, 12....","[10.97, 10.97, 11.02, 11.02, 11.04, 11.04, 10....","[1008.1, 1008.1, 1007.88, 1007.88, 1007.95, 10...","[12.12, 12.12, 12.59, 12.59, 12.45, 12.45, 12....","[5.9, 5.9, 6.06, 6.06, 5.31, 5.31, 4.68, 4.68,...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, clo...","[0.93, 0.93, 0.9, 0.9, 0.91, 0.91, 0.93, 0.93,...","[Mostly Cloudy, Mostly Cloudy, Overcast, Overc..."
630,MAC002364,2012-04-28,30min,"[0.012, 0.027, 0.0, 0.038, 0.0, 0.035, 0.003, ...",32208,Std,ACORN-Q,Adversity,block_109,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[41, 41, 26, 26, 31, 31, 32, 32, 30, 30, 35, 3...","[9.59, 9.59, 9.46, 9.46, 9.12, 9.12, 8.71, 8.7...","[7.93, 7.93, 7.9, 7.9, 7.64, 7.64, 7.71, 7.71,...","[1015.2, 1015.2, 1015.27, 1015.27, 1014.94, 10...","[9.33, 9.33, 8.59, 8.59, 7.76, 7.76, 6.87, 6.8...","[1.37, 1.37, 1.97, 1.97, 2.52, 2.52, 3.15, 3.1...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.89, 0.89, 0.9, 0.9, 0.9, 0.9, 0.93, 0.93, 0...","[Partly Cloudy, Partly Cloudy, Partly Cloudy, ..."


In [18]:
# fill in missing values
from src.imputation.interpolation import SeasonalInterpolation

block_df.energy_consumption = block_df.energy_consumption.progress_apply\
    (lambda x: SeasonalInterpolation(seasonal_period=48*7).fit_transform(np.array(x).reshape(-1,1)).squeeze())

  0%|          | 0/30 [00:00<?, ?it/s]

In [21]:
# deseasonalize and detrend

from src.decomposition.seasonal import MultiSeasonalDecomposition

def make_stationary(row):
#     print(row)
    # Order of row: LCLid, timestamp, frequency, energy_consumption
    ts = row[3]
    dates = pd.date_range(start=row[1], freq=row[2], periods=len(ts))
    stl = MultiSeasonalDecomposition(seasonal_model="fourier",seasonality_periods=["day_of_year", "day_of_week", "hour"], model = "additive", n_fourier_terms=10)
    res = stl.fit(pd.Series(ts, index=dates))
    return res.resid.values# + res.trend.values

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning) 
    block_df['residuals'] =[make_stationary(r) for r in tqdm(zip(*block_df.to_dict("list").values()), total=len(block_df))]

  0%|          | 0/30 [00:00<?, ?it/s]

coefficient of variation (CoV)

In [22]:
# the more variability you find in a ts, the harder it is to predict it

# CoV_n (for ts n) = sigma_n / mu_n 
#   the relative dispersion of data points around the mean, which is better than looking at pure standard deviation
#   the larger the cov, the worse the predictability of ts
#   0.49 is rule of thumb to separate ts that easier to forecast from the hard ones
# CoV's key issues: 
#   it doesn't consider seasonality (e.g. sine/consine waves show higher CoV but are equally predictable)
#   it doesn't consider trend (e.g. linear trend has higher CoV but is equally predictable)
#   doesn't handle negative values (e.g. -ve in ts makes mean smaller and inflate CoV)

from src.forecastability.cov import calc_cov

block_df["cov"] = block_df.progress_apply(lambda x: calc_cov(x['energy_consumption']), axis=1)

# plot
fig = px.histogram(x=block_df["cov"], title="Distribution of Coefficient of Variation")
fig.show()

  0%|          | 0/30 [00:00<?, ?it/s]

residual variability

In [23]:
# - perform seasonal decomposition
# - calculate the standard deviation of the residuals
# - divde the sd by the mean of the original observed values (before decomposition)

# key assumption is that seasonality and trend are components that can be predicted

from src.forecastability.cov import calc_norm_sd

block_df["residual_variability"] = block_df.progress_apply\
    (lambda x: calc_norm_sd(x['residuals'],x['energy_consumption']), axis=1)

# plot
fig = px.histogram(x=block_df["residual_variability"], title="Distribution of Residual Variability")
fig.show()

  0%|          | 0/30 [00:00<?, ?it/s]

spectral entropy

In [24]:
# entropy is formally defined as H(X) = - sum{P(x_i)*log(P(x_i))}
#          - X is the discrete random variable with possible outcomes x_1, x_2, ..., x_n
#          - each outcome has prob of occuring by P(x_1), P(x_2), ..., P(x_n)

# spectral entropy is calucated by discretization of the continuous time series (Fast Fourier Transform FFT and power spectral density PSD)
#          - nPSD_i = PSD_i / sum_{j=1}^{F}{PSD_j}      F is the # of freq that are part of the returned PSD
#          - H_s(X) = - sum{nPSD_i * log(nPSD_i)}
 
from src.forecastability.entropy import spectral_entropy

block_df["residual_spectral_entropy"] = block_df.residuals.progress_apply(spectral_entropy)
block_df["spectral_entropy"] = block_df.energy_consumption.progress_apply(lambda x: spectral_entropy(x, transform_stationary=True))

# plot
fig = px.histogram(x=block_df["spectral_entropy"], title="Distribution of Spectral Entropy")
fig.show()

fig = px.histogram(x=block_df["residual_spectral_entropy"], title="Distribution of Residual Spectral Entropy")
fig.show()

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

kaboudan metric

In [25]:
# kaboudan metric = 1 - SSE_Y/SSE_S
#          - SSE_Y: sum of squared error of the forecast that was generated from the original ts
#          - SSE_S: sum of sqaured error of the forecast that was generated from the block-shuffled series
#          - kaboudan metric -> 1 (if ts has signals as SSE_Y<SSE_S)
#          - kaboundan metric -> 0 even -ve (if ts unpredictable or even just white noise as SSE_Y ~ SSE_S)

from src.forecastability.kaboudan import kaboudan_metric

block_size = 5
freq = '30min'
models = [Theta(season_length=48*7, decomposition_type='additive')]

 
block_df["kaboudan_metric"] = [kaboudan_metric(r[0], 
                                               model=models[0], 
                                               block_size=block_size,  
                                               backtesting_start=0.5, 
                                               n_folds=1,
                                               freq = freq) 
                                               for r in tqdm(zip(*block_df[["energy_consumption"]].to_dict("list").values()), total=len(block_df))]

block_df[['LCLid','kaboudan_metric']].head()

  0%|          | 0/30 [00:00<?, ?it/s]

,LCLid,kaboudan_metric
3916,MAC000193,-4.923346
630,MAC002364,0.741978
1014,MAC000321,-1.140900
1049,MAC001979,-0.129189
1108,MAC005521,-0.125817


In [26]:
# modified kaboudan metric = 1 - sqrt(SSE_Y/SSE_S)

from src.forecastability.kaboudan import modified_kaboudan_metric 

block_df["modified_kaboudan_metric"] = [modified_kaboudan_metric(r[0], 
                                               model=models[0], 
                                               block_size=block_size,  
                                               backtesting_start=0.5, 
                                               n_folds=1,
                                               freq = freq) 
                                               for r in tqdm(zip(*block_df[["energy_consumption"]].to_dict("list").values()), total=len(block_df))]

block_df[['LCLid','modified_kaboudan_metric']].head()

  0%|          | 0/30 [00:00<?, ?it/s]

,LCLid,modified_kaboudan_metric
3916,MAC000193,0.000000
630,MAC002364,0.424711
1014,MAC000321,0.061647
1049,MAC001979,0.000000
1108,MAC005521,0.000000
